In [73]:
import pandas_datareader.data as web
import datetime
from decimal import Decimal
from dateutil import relativedelta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, resample_apply
import talib as ta
import importlib
import os
import platform
import psutil

#https://www.sbisec.co.jp/ETGate/WPLETmgR001Control?OutSide=on&getFlg=on&burl=search_domestic&cat1=domestic&cat2=none&dir=info&file=domestic_info150818_01.html

# start = datetime.date(2021,1,1)
start = datetime.datetime.today() + relativedelta.relativedelta(years=-1)
end = datetime.date.today()
# data = web.DataReader('7203.T', 'yahoo', start, end)
# data = web.DataReader('SPYD', 'yahoo', start, end)
# data = web.DataReader('1321.T', 'yahoo', start, end)
# data = web.DataReader('1570.T', 'yahoo', start, end)  #NEXT FUNDS 日経平均レバレッジ・インデックス連動型上場投信
# data = web.DataReader('1357.T', 'yahoo', start, end)  #NEXT FUNDS 日経平均ダブルインバース・インデックス連動型上場投信
data = web.DataReader('BTC-JPY', 'yahoo', start, end)
# data = web.DataReader('AAPL', 'yahoo', start, end)
data = data.astype("double")
print(data)

initial = 10000000
fname = f"html/plot-"

#position
#0 ... display "only long" result
#1 ... display both "only long" and "long and short" result
position = 1

                  High         Low       Open       Close        Volume  \
Date                                                                      
2021-09-14  5181479.50  4921861.50  4945451.5  5166778.00  4.240741e+12   
2021-09-15  5296968.50  5123403.50  5167382.0  5269729.00  3.334521e+12   
2021-09-16  5303107.00  5166541.00  5267821.5  5241810.50  3.484527e+12   
2021-09-17  5290284.50  5149142.50  5240455.0  5196071.00  3.158009e+12   
2021-09-18  5363631.50  5176258.00  5196731.5  5307192.00  3.141290e+12   
...                ...         ...        ...         ...           ...   
2022-09-09  3055224.25  2779062.75  2780884.5  3046922.25  6.907153e+12   
2022-09-10  3100949.25  3016649.75  3046318.0  3089586.50  5.260394e+12   
2022-09-11  3103808.25  3050597.75  3089302.0  3103623.25  4.917772e+12   
2022-09-12  3198483.25  3088417.25  3103750.5  3189668.00  7.159440e+12   
2022-09-14  2953935.75  2895084.50  2911983.0  2895084.50  7.006261e+12   

             Adj Close  

**MACD CROSS**

In [74]:
import BacktestingStrategies.Strategy_MacdCross as btstMACD
importlib.reload(btstMACD)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstMACD.MACDCross, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True, #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + ".html") # 実行結果（グラフ）

#最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)
output2=bt.optimize(\
    MACDshort=range(10, 25, 5),\
    MACDlong=range(10, 75, 10),\
    MACDsignal=range(10, 15, 5),\
    maximize='Equity Final [$]',
    method='grid',\
    constraint = lambda x: (x.MACDlong > x.MACDshort) & (x.MACDshort > x.MACDsignal),\
    max_tries = 1000)
print(output2)
display(output2._trades)
display(output2._strategy)
bt.plot(filename = fname + str(output._strategy) + "_optimize.html") 


if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstMACD.MACDCross_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True, #自動でポジションをクローズ(オープン)
    )

    output = bt.run() # バックテスト実行
    print(output) # 実行結果(データ)
    display(output._trades)
    bt.plot(filename = fname + str(output._strategy) + ".html") # 実行結果（グラフ）

    #最適化
    p = psutil.Process()
    p.nice(psutil.HIGH_PRIORITY_CLASS)
    output2=bt.optimize(\
        MACDshort=range(10, 25, 5),\
        MACDlong=range(10, 75, 10),\
        MACDsignal=range(10, 15, 5),\
        maximize='Equity Final [$]',
        method='grid',\
        constraint = lambda x: (x.MACDlong > x.MACDshort) & (x.MACDshort > x.MACDsignal),\
        max_tries = 1000)
    print(output2)
    display(output2._trades)
    display(output2._strategy)
    bt.plot(filename = fname + str(output._strategy) + "_optimize.html") 


Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   42.739726
Equity Final [$]                 7928485.3602
Equity Peak [$]               10485934.062975
Return [%]                         -20.715146
Buy & Hold Return [%]              -43.967314
Return (Ann.) [%]                  -20.715146
Volatility (Ann.) [%]               27.082932
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -33.903472
Avg. Drawdown [%]                  -17.090121
Max. Drawdown Duration      264 days 00:00:00
Avg. Drawdown Duration      133 days 00:00:00
# Trades                                   10
Win Rate [%]                             40.0
Best Trade [%]                      19.514892
Worst Trade [%]                    -22.510517
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,1,97,114,5.328455e+06,5059130.00,-2.693254e+05,-0.050545,2021-12-20,2022-01-06,17 days
1,1,124,130,4.956170e+06,4146638.25,-8.095320e+05,-0.163338,2022-01-16,2022-01-22,6 days
2,2,138,158,4.418967e+06,4602393.00,3.668517e+05,0.041509,2022-01-30,2022-02-19,20 days
3,1,168,174,4.995646e+06,4417545.00,-5.781011e+05,-0.115721,2022-03-01,2022-03-07,6 days
4,1,177,178,4.889340e+06,4582254.00,-3.070855e+05,-0.062807,2022-03-10,2022-03-11,1 days
5,1,182,204,4.712339e+06,5631946.50,9.196078e+05,0.195149,2022-03-15,2022-04-06,22 days
6,2,251,273,3.895487e+06,3018592.50,-1.753788e+06,-0.225105,2022-05-23,2022-06-14,22 days
7,2,285,324,2.920928e+06,3060081.00,2.783052e+05,0.047640,2022-06-26,2022-08-04,39 days
8,2,329,338,3.226574e+06,3150017.25,-1.531132e+05,-0.023727,2022-08-09,2022-08-18,9 days
9,2,360,364,2.794650e+06,2911983.00,2.346662e+05,0.041985,2022-09-09,2022-09-14,5 days


Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   42.739726
Equity Final [$]               7785760.297025
Equity Peak [$]               10460028.461875
Return [%]                         -22.142397
Buy & Hold Return [%]              -43.967314
Return (Ann.) [%]                  -22.142397
Volatility (Ann.) [%]               27.102737
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -35.104256
Avg. Drawdown [%]                  -11.967767
Max. Drawdown Duration      264 days 00:00:00
Avg. Drawdown Duration       90 days 00:00:00
# Trades                                   10
Win Rate [%]                             40.0
Best Trade [%]                      14.302938
Worst Trade [%]                    -22.510517
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,1,96,114,5.354361e+06,5059130.00,-2.952310e+05,-0.055138,2021-12-19,2022-01-06,18 days
1,1,124,130,4.956170e+06,4146638.25,-8.095320e+05,-0.163338,2022-01-16,2022-01-22,6 days
2,2,138,158,4.418967e+06,4602393.00,3.668517e+05,0.041509,2022-01-30,2022-02-19,20 days
3,1,168,174,4.995646e+06,4417545.00,-5.781011e+05,-0.115721,2022-03-01,2022-03-07,6 days
4,1,177,178,4.889340e+06,4582254.00,-3.070855e+05,-0.062807,2022-03-10,2022-03-11,1 days
5,1,183,205,4.676959e+06,5345902.00,6.689426e+05,0.143029,2022-03-16,2022-04-07,22 days
6,2,251,273,3.895487e+06,3018592.50,-1.753788e+06,-0.225105,2022-05-23,2022-06-14,22 days
7,2,286,324,2.854005e+06,3060081.00,4.121510e+05,0.072206,2022-06-27,2022-08-04,38 days
8,2,329,338,3.226574e+06,3150017.25,-1.531132e+05,-0.023727,2022-08-09,2022-08-18,9 days
9,2,360,364,2.794650e+06,2911983.00,2.346662e+05,0.041985,2022-09-09,2022-09-14,5 days


<Strategy MACDCross(MACDshort=15,MACDlong=20,MACDsignal=10)>

Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   89.041096
Equity Final [$]                9179018.61565
Equity Peak [$]               12605861.123825
Return [%]                          -8.209814
Buy & Hold Return [%]              -43.967314
Return (Ann.) [%]                   -8.209814
Volatility (Ann.) [%]                45.32877
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -43.039208
Avg. Drawdown [%]                   -6.589468
Max. Drawdown Duration      264 days 00:00:00
Avg. Drawdown Duration       30 days 00:00:00
# Trades                                   20
Win Rate [%]                             45.0
Best Trade [%]                      30.830626
Worst Trade [%]                    -22.510517
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-1,40,97,6.936202e+06,5302209.50,1.633993e+06,0.235575,2021-10-24,2021-12-20,57 days
1,2,97,114,5.328455e+06,5059130.00,-5.386509e+05,-0.050545,2021-12-20,2022-01-06,17 days
2,-2,114,124,5.034087e+06,4931758.00,2.046586e+05,0.020327,2022-01-06,2022-01-16,10 days
3,2,124,130,4.956170e+06,4146638.25,-1.619064e+06,-0.163338,2022-01-16,2022-01-22,6 days
4,-2,130,138,4.126112e+06,4397201.00,-5.421772e+05,-0.065701,2022-01-22,2022-01-30,8 days
5,2,138,158,4.418967e+06,4602393.00,3.668517e+05,0.041509,2022-01-30,2022-02-19,20 days
6,-2,158,168,4.579611e+06,4971039.50,-7.828567e+05,-0.085472,2022-02-19,2022-03-01,10 days
7,1,168,174,4.995646e+06,4417545.00,-5.781011e+05,-0.115721,2022-03-01,2022-03-07,6 days
8,-1,174,177,4.395678e+06,4865256.50,-4.695783e+05,-0.106827,2022-03-07,2022-03-10,3 days
9,1,177,178,4.889340e+06,4582254.00,-3.070855e+05,-0.062807,2022-03-10,2022-03-11,1 days


Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   88.767123
Equity Final [$]               9418588.767575
Equity Peak [$]                11725478.84605
Return [%]                          -5.814112
Buy & Hold Return [%]              -43.967314
Return (Ann.) [%]                   -5.814112
Volatility (Ann.) [%]               48.183298
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -36.256563
Avg. Drawdown [%]                   -6.886358
Max. Drawdown Duration      264 days 00:00:00
Avg. Drawdown Duration       33 days 00:00:00
# Trades                                   18
Win Rate [%]                        44.444444
Best Trade [%]                      29.350878
Worst Trade [%]                    -20.074131
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-1,41,99,6.887626e+06,5578566.0,1.309060e+06,0.190060,2021-10-25,2021-12-22,58 days
1,2,99,114,5.606180e+06,5059130.0,-1.094100e+06,-0.097580,2021-12-22,2022-01-06,15 days
2,-2,114,125,5.034087e+06,4928954.5,2.102656e+05,0.020884,2022-01-06,2022-01-17,11 days
3,2,125,129,4.953353e+06,4639416.0,-6.278736e+05,-0.063379,2022-01-17,2022-01-21,4 days
4,-2,129,138,4.616451e+06,4397201.0,4.384998e+05,0.047493,2022-01-21,2022-01-30,9 days
5,2,138,160,4.418967e+06,4417142.0,-3.650290e+03,-0.000413,2022-01-30,2022-02-21,22 days
6,-2,160,168,4.395277e+06,4971039.5,-1.151525e+06,-0.130996,2022-02-21,2022-03-01,8 days
7,1,168,174,4.995646e+06,4417545.0,-5.781011e+05,-0.115721,2022-03-01,2022-03-07,6 days
8,-1,174,177,4.395678e+06,4865256.5,-4.695783e+05,-0.106827,2022-03-07,2022-03-10,3 days
9,1,177,179,4.889340e+06,4550126.0,-3.392135e+05,-0.069378,2022-03-10,2022-03-12,2 days


<Strategy MACDCross_WithShortPosition(MACDshort=15,MACDlong=30,MACDsignal=10)>

**EMA CROSS**

In [75]:
import BacktestingStrategies.Strategy_EMACross as btstEMA
importlib.reload(btstEMA)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstEMA.EmaCrossStrategy, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + ".html") 

#最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)

output2=bt.optimize(\
    EMAshort=range(10, 100, 10),\
    EMAlong=range(10, 200, 5),\
    maximize='Equity Final [$]',\
    method='grid',\
    constraint = lambda x: (x.EMAlong > x.EMAshort),\
    max_tries = 1000)
print(output2)
display(output2._trades)
display(output2._strategy)
bt.plot(filename = fname + str(output._strategy) + "_optimize.html")

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstEMA.EmaCrossStrategy_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )

    output = bt.run() # バックテスト実行
    print(output) # 実行結果(データ)
    display(output._trades)
    bt.plot(filename = fname + str(output._strategy) + ".html") 

    #最適化
    p = psutil.Process()
    p.nice(psutil.HIGH_PRIORITY_CLASS)

    output2=bt.optimize(\
        EMAshort=range(10, 100, 10),\
        EMAlong=range(10, 200, 5),\
        maximize='Equity Final [$]',\
        method='grid',\
        constraint = lambda x: (x.EMAlong > x.EMAshort),\
        max_tries = 1000)
    print(output2)
    display(output2._trades)
    display(output2._strategy)
    bt.plot(filename = fname + str(output._strategy) + "_optimize.html")

Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   44.657534
Equity Final [$]               7488524.873225
Equity Peak [$]               12175429.139575
Return [%]                         -25.114751
Buy & Hold Return [%]              -43.967314
Return (Ann.) [%]                  -25.114751
Volatility (Ann.) [%]               19.203862
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -38.494777
Avg. Drawdown [%]                   -8.558544
Max. Drawdown Duration      329 days 00:00:00
Avg. Drawdown Duration       69 days 00:00:00
# Trades                                   16
Win Rate [%]                            18.75
Best Trade [%]                      23.800965
Worst Trade [%]                    -10.587818
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,1,18,44,5.373409e+06,6652332.00,1.278923e+06,0.238010,2021-10-02,2021-10-28,26 days
1,1,46,54,7.131348e+06,6981251.50,-1.500963e+05,-0.021047,2021-10-30,2021-11-07,8 days
2,1,55,64,7.223739e+06,6910341.00,-3.133984e+05,-0.043385,2021-11-08,2021-11-17,9 days
3,1,101,107,5.844281e+06,5344083.50,-5.001972e+05,-0.085587,2021-12-24,2021-12-30,6 days
4,2,141,142,4.467342e+06,4225814.50,-4.830548e+05,-0.054065,2022-02-02,2022-02-03,1 days
5,2,144,158,4.805679e+06,4602393.00,-4.065719e+05,-0.042301,2022-02-05,2022-02-19,14 days
6,1,168,173,4.995646e+06,4524193.00,-4.714531e+05,-0.094373,2022-03-01,2022-03-06,5 days
7,1,177,178,4.889340e+06,4582254.00,-3.070855e+05,-0.062807,2022-03-10,2022-03-11,1 days
8,1,183,205,4.676959e+06,5345902.00,6.689426e+05,0.143029,2022-03-16,2022-04-07,22 days
9,1,218,221,5.394688e+06,5104440.00,-2.902477e+05,-0.053802,2022-04-20,2022-04-23,3 days


Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   10.136986
Equity Final [$]                9829866.95765
Equity Peak [$]                11815235.95765
Return [%]                           -1.70133
Buy & Hold Return [%]              -43.967314
Return (Ann.) [%]                    -1.70133
Volatility (Ann.) [%]               15.461132
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -16.809584
Avg. Drawdown [%]                  -16.809584
Max. Drawdown Duration      169 days 00:00:00
Avg. Drawdown Duration      169 days 00:00:00
# Trades                                    1
Win Rate [%]                              0.0
Best Trade [%]                      -1.723201
Worst Trade [%]                     -1.723201
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,2,189,225,4.936542e+06,4851475.5,-170133.04235,-0.017232,2022-03-22,2022-04-27,36 days


<Strategy EmaCrossStrategy(EMAshort=10,EMAlong=70)>

Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   95.068493
Equity Final [$]               7843087.894037
Equity Peak [$]               13327319.912825
Return [%]                         -21.569121
Buy & Hold Return [%]              -43.967314
Return (Ann.) [%]                  -21.569121
Volatility (Ann.) [%]               37.456847
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -41.150299
Avg. Drawdown [%]                  -10.443765
Max. Drawdown Duration      235 days 00:00:00
Avg. Drawdown Duration       57 days 00:00:00
# Trades                                   31
Win Rate [%]                        25.806452
Best Trade [%]                      23.800965
Worst Trade [%]                    -13.724743
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,1,18,44,5.373409e+06,6652332.00,1.278923e+06,0.238010,2021-10-02,2021-10-28,26 days
1,-1,44,46,6.619403e+06,7096221.50,-4.768185e+05,-0.072033,2021-10-28,2021-10-30,2 days
2,1,46,54,7.131348e+06,6981251.50,-1.500963e+05,-0.021047,2021-10-30,2021-11-07,8 days
3,-1,54,55,6.946694e+06,7188158.00,-2.414637e+05,-0.034760,2021-11-07,2021-11-08,1 days
4,1,55,64,7.223739e+06,6910341.00,-3.133984e+05,-0.043385,2021-11-08,2021-11-17,9 days
5,-1,64,101,6.876135e+06,5815494.00,1.060641e+06,0.154250,2021-11-17,2021-12-24,37 days
6,1,101,107,5.844281e+06,5344083.50,-5.001972e+05,-0.085587,2021-12-24,2021-12-30,6 days
7,-2,107,141,5.317630e+06,4445337.50,1.744586e+06,0.164038,2021-12-30,2022-02-02,34 days
8,2,141,142,4.467342e+06,4225814.50,-4.830548e+05,-0.054065,2022-02-02,2022-02-03,1 days
9,-2,142,144,4.204897e+06,4782008.00,-1.154223e+06,-0.137247,2022-02-03,2022-02-05,2 days


Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   78.082192
Equity Final [$]               14353628.15105
Equity Peak [$]                 16193317.3007
Return [%]                          43.536282
Buy & Hold Return [%]              -43.967314
Return (Ann.) [%]                   43.536282
Volatility (Ann.) [%]                51.53951
Sharpe Ratio                         0.844717
Sortino Ratio                        1.940947
Calmar Ratio                         2.389472
Max. Drawdown [%]                  -18.220042
Avg. Drawdown [%]                    -5.43683
Max. Drawdown Duration       88 days 00:00:00
Avg. Drawdown Duration       20 days 00:00:00
# Trades                                    5
Win Rate [%]                             60.0
Best Trade [%]                      37.563386
Worst Trade [%]                     -6.327056
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-1,80,187,6.355164e+06,5029849.00,1.325315e+06,0.208541,2021-12-03,2022-03-20,107 days
1,2,187,226,5.054747e+06,5040468.50,-2.855651e+04,-0.002825,2022-03-20,2022-04-28,39 days
2,-2,226,330,5.015518e+06,3131519.75,3.767997e+06,0.375634,2022-04-28,2022-08-10,104 days
3,4,330,342,3.147021e+06,2947907.00,-7.964551e+05,-0.063271,2022-08-10,2022-08-22,12 days
4,-4,342,364,2.933315e+06,2911983.00,8.532744e+04,0.007272,2022-08-22,2022-09-14,23 days


<Strategy EmaCrossStrategy_WithShortPosition(EMAshort=20,EMAlong=40)>

**SMA**

In [76]:
import BacktestingStrategies.Strategy_SMACross as btstSMA
importlib.reload(btstSMA)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstSMA.SmaCross, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + ".html")

# #最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)

output2=bt.optimize(\
    SMA_short=range(10, 100, 5),\
    SMA_long=range(10, 200, 5),\
    maximize='Equity Final [$]',\
    method='grid',\
    max_tries = 1000,\
    constraint = lambda x: x.SMA_short < x.SMA_long)
print(output2)
display(output2._trades)
display(output2._strategy)
bt.plot(filename = fname + str(output._strategy) + "_optimize.html")

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstSMA.SmaCross_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )

    output = bt.run() # バックテスト実行
    print(output) # 実行結果(データ)
    display(output._trades)
    bt.plot(filename = fname + str(output._strategy) + ".html")

    # #最適化
    p = psutil.Process()
    p.nice(psutil.HIGH_PRIORITY_CLASS)

    output2=bt.optimize(\
        SMA_short=range(10, 100, 5),\
        SMA_long=range(10, 200, 5),\
        maximize='Equity Final [$]',\
        method='grid',\
        max_tries = 1000,\
        constraint = lambda x: x.SMA_short < x.SMA_long)
    print(output2)
    display(output2._trades)
    display(output2._strategy)
    bt.plot(filename = fname + str(output._strategy) + "_optimize.html") 

Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   28.493151
Equity Final [$]                7352085.00085
Equity Peak [$]                10034663.86515
Return [%]                          -26.47915
Buy & Hold Return [%]              -43.967314
Return (Ann.) [%]                   -26.47915
Volatility (Ann.) [%]               18.493463
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -31.861801
Avg. Drawdown [%]                  -16.552036
Max. Drawdown Duration      256 days 00:00:00
Avg. Drawdown Duration      129 days 00:00:00
# Trades                                    6
Win Rate [%]                        33.333333
Best Trade [%]                      10.868693
Worst Trade [%]                    -27.933512
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,1,108,113,5.454470e+06,5332381.0,-1.220886e+05,-0.022383,2021-12-31,2022-01-05,5 days
1,1,148,164,5.120042e+06,4429464.5,-6.905779e+05,-0.134877,2022-02-09,2022-02-25,16 days
2,1,177,178,4.889340e+06,4582254.0,-3.070855e+05,-0.062807,2022-03-10,2022-03-11,1 days
3,1,185,212,4.875599e+06,5160874.0,2.852747e+05,0.058511,2022-03-18,2022-04-14,27 days
4,2,266,274,4.167187e+06,3003145.5,-2.328083e+06,-0.279335,2022-06-07,2022-06-15,8 days
5,2,302,343,2.655662e+06,2944297.5,5.772714e+05,0.108687,2022-07-13,2022-08-23,41 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 513 configurations.
  output = _optimize_grid()


Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                    3.287671
Equity Final [$]               10085375.74095
Equity Peak [$]                10085506.24095
Return [%]                           0.853757
Buy & Hold Return [%]              -43.967314
Return (Ann.) [%]                    0.853757
Volatility (Ann.) [%]                4.583061
Sharpe Ratio                         0.186285
Sortino Ratio                        0.253917
Calmar Ratio                         0.238224
Max. Drawdown [%]                    -3.58385
Avg. Drawdown [%]                   -1.792572
Max. Drawdown Duration      148 days 00:00:00
Avg. Drawdown Duration       79 days 00:00:00
# Trades                                    1
Win Rate [%]                            100.0
Best Trade [%]                       1.616126
Worst Trade [%]                      1.616126
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,1,207,218,5.282740e+06,5368115.5,85375.74095,0.016161,2022-04-09,2022-04-20,11 days


...

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-1,68,108,6.775275e+06,5427603.00,1.347672e+06,0.198910,2021-11-21,2021-12-31,40 days
1,2,108,113,5.454470e+06,5332381.00,-2.441773e+05,-0.022383,2021-12-31,2022-01-05,5 days
2,-2,113,148,5.305986e+06,5094823.00,4.223254e+05,0.039797,2022-01-05,2022-02-09,35 days
3,2,148,164,5.120042e+06,4429464.50,-1.381156e+06,-0.134877,2022-02-09,2022-02-25,16 days
4,-2,164,177,4.407539e+06,4865256.50,-9.154357e+05,-0.103849,2022-02-25,2022-03-10,13 days
5,1,177,178,4.889340e+06,4582254.00,-3.070855e+05,-0.062807,2022-03-10,2022-03-11,1 days
6,-1,178,185,4.559572e+06,4851584.00,-2.920122e+05,-0.064044,2022-03-11,2022-03-18,7 days
7,1,185,212,4.875599e+06,5160874.00,2.852747e+05,0.058511,2022-03-18,2022-04-14,27 days
8,-1,212,266,5.135328e+06,4146661.25,9.886664e+05,0.192523,2022-04-14,2022-06-07,54 days
9,2,266,274,4.167187e+06,3003145.50,-2.328083e+06,-0.279335,2022-06-07,2022-06-15,8 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 513 configurations.
  output = _optimize_grid()


Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   80.547945
Equity Final [$]              15476779.993375
Equity Peak [$]               18073283.069625
Return [%]                            54.7678
Buy & Hold Return [%]              -43.967314
Return (Ann.) [%]                     54.7678
Volatility (Ann.) [%]                64.87203
Sharpe Ratio                         0.844244
Sortino Ratio                        2.161831
Calmar Ratio                         2.483944
Max. Drawdown [%]                  -22.048723
Avg. Drawdown [%]                   -5.721484
Max. Drawdown Duration       88 days 00:00:00
Avg. Drawdown Duration       16 days 00:00:00
# Trades                                    7
Win Rate [%]                        71.428571
Best Trade [%]                      39.794413
Worst Trade [%]                     -8.742387
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-1,71,152,6.595423e+06,4875155.5,1.720267e+06,0.260827,2021-11-24,2022-02-13,81 days
1,2,152,171,4.899288e+06,4903207.5,7.839961e+03,0.000800,2022-02-13,2022-03-04,19 days
2,-2,171,189,4.878937e+06,4912226.5,-6.657975e+04,-0.006823,2022-03-04,2022-03-22,18 days
3,2,189,218,4.936542e+06,5368115.5,8.631470e+05,0.087424,2022-03-22,2022-04-20,29 days
4,-2,218,310,5.341543e+06,3215907.5,4.251272e+06,0.397944,2022-04-20,2022-07-21,92 days
5,5,310,346,3.231826e+06,2949287.5,-1.412694e+06,-0.087424,2022-07-21,2022-08-26,36 days
6,-5,346,364,2.934689e+06,2911983.0,1.135276e+05,0.007737,2022-08-26,2022-09-14,19 days


<Strategy SmaCross_WithShortPosition(SMA_short=15,SMA_long=40)>

**ボリンジャーバンド**

In [77]:
import BacktestingStrategies.Strategy_BBD as btstBBD
importlib.reload(btstBBD)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstBBD.BBsigma, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + ".html")

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstBBD.BBsigma_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )

    output = bt.run() # バックテスト実行
    print(output) # 実行結果(データ)
    display(output._trades)
    bt.plot(filename = fname + str(output._strategy) + ".html")


Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   60.547945
Equity Final [$]               7648983.129025
Equity Peak [$]                10237876.08275
Return [%]                         -23.510169
Buy & Hold Return [%]              -43.967314
Return (Ann.) [%]                  -23.510169
Volatility (Ann.) [%]               27.612573
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -34.709747
Avg. Drawdown [%]                  -34.709747
Max. Drawdown Duration      298 days 00:00:00
Avg. Drawdown Duration      298 days 00:00:00
# Trades                                   35
Win Rate [%]                        42.857143
Best Trade [%]                      29.669562
Worst Trade [%]                    -14.465488
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,1,66,67,6.534513e+06,6624828.50,9.031548e+04,0.013821,2021-11-19,2021-11-20,1 days
1,1,67,74,6.657621e+06,6089680.00,-5.679414e+05,-0.085307,2021-11-20,2021-11-27,7 days
2,1,74,75,6.119824e+06,6211685.00,9.186108e+04,0.015010,2021-11-27,2021-11-28,1 days
3,1,75,82,6.242433e+06,5550177.50,-6.922553e+05,-0.110895,2021-11-28,2021-12-05,7 days
4,1,82,83,5.577651e+06,5585922.00,8.271121e+03,0.001483,2021-12-05,2021-12-06,1 days
5,1,83,114,5.613572e+06,5059130.00,-5.544423e+05,-0.098768,2021-12-06,2022-01-06,31 days
6,1,114,115,5.084173e+06,5000048.50,-8.412419e+04,-0.016546,2022-01-06,2022-01-07,1 days
7,1,115,116,5.024799e+06,4801785.00,-2.230137e+05,-0.044383,2022-01-07,2022-01-08,1 days
8,1,116,117,4.825554e+06,4824033.50,-1.520336e+03,-0.000315,2022-01-08,2022-01-09,1 days
9,1,117,130,4.847912e+06,4146638.25,-7.012742e+05,-0.144655,2022-01-09,2022-01-22,13 days


Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   92.328767
Equity Final [$]               9216664.449662
Equity Peak [$]                10127837.48165
Return [%]                          -7.833356
Buy & Hold Return [%]              -43.967314
Return (Ann.) [%]                   -7.833356
Volatility (Ann.) [%]               41.716498
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -30.294875
Avg. Drawdown [%]                  -30.294875
Max. Drawdown Duration      337 days 00:00:00
Avg. Drawdown Duration      337 days 00:00:00
# Trades                                   50
Win Rate [%]                             44.0
Best Trade [%]                      29.669562
Worst Trade [%]                    -14.465488
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-1,28,32,6.490644e+06,7046928.50,-5.562840e+05,-0.085706,2021-10-12,2021-10-16,4 days
1,-1,32,56,7.012046e+06,7650345.50,-6.382993e+05,-0.091029,2021-10-16,2021-11-09,24 days
2,-1,56,57,7.612476e+06,7560003.00,5.247329e+04,0.006893,2021-11-09,2021-11-10,1 days
3,-1,57,66,7.522581e+06,6502326.50,1.020254e+06,0.135626,2021-11-10,2021-11-19,9 days
4,1,66,67,6.534513e+06,6624828.50,9.031548e+04,0.013821,2021-11-19,2021-11-20,1 days
5,1,67,74,6.657621e+06,6089680.00,-5.679414e+05,-0.085307,2021-11-20,2021-11-27,7 days
6,1,74,75,6.119824e+06,6211685.00,9.186108e+04,0.015010,2021-11-27,2021-11-28,1 days
7,1,75,82,6.242433e+06,5550177.50,-6.922553e+05,-0.110895,2021-11-28,2021-12-05,7 days
8,1,82,83,5.577651e+06,5585922.00,8.271121e+03,0.001483,2021-12-05,2021-12-06,1 days
9,1,83,114,5.613572e+06,5059130.00,-5.544423e+05,-0.098768,2021-12-06,2022-01-06,31 days


**RSI**

In [78]:
import BacktestingStrategies.Strategy_RsiOscillator as btstRsi
importlib.reload(btstRsi)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstRsi.RsiOscillator, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + ".html")

# #最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)

output2=bt.optimize(\
    upper_bound = range(50, 85, 5),
    lower_bound = range(10, 45, 5),
    rsi_window=range(10, 30, 2),\
    maximize='Equity Final [$]',\
    method='grid',\
    max_tries = 1000)
print(output2)
display(output2._trades)
display(output2._strategy)
bt.plot(filename = fname + str(output._strategy) + "_optimize.html") 

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstRsi.RsiOscillator_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )

    output = bt.run() # バックテスト実行
    print(output) # 実行結果(データ)
    display(output._trades)
    bt.plot(filename = fname + str(output._strategy) + ".html")

    # #最適化
    p = psutil.Process()
    p.nice(psutil.HIGH_PRIORITY_CLASS)

    output2=bt.optimize(\
        upper_bound = range(50, 85, 5),
        lower_bound = range(10, 45, 5),
        rsi_window=range(10, 30, 2),\
        maximize='Equity Final [$]',\
        method='grid',\
        max_tries = 1000)
    print(output2)
    display(output2._trades)
    display(output2._strategy)
    bt.plot(filename = fname + str(output._strategy) + "_optimize.html")

Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   56.164384
Equity Final [$]               9418797.735375
Equity Peak [$]                11839816.66335
Return [%]                          -5.812023
Buy & Hold Return [%]              -43.967314
Return (Ann.) [%]                   -5.812023
Volatility (Ann.) [%]               46.192015
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -27.948686
Avg. Drawdown [%]                  -11.708353
Max. Drawdown Duration      100 days 00:00:00
Avg. Drawdown Duration       29 days 00:00:00
# Trades                                    6
Win Rate [%]                             50.0
Best Trade [%]                      35.363142
Worst Trade [%]                    -28.844867
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,2,117,134,4.847912e+06,4208502.00,-1.278821e+06,-0.131894,2022-01-09,2022-01-26,17 days
1,2,134,195,4.229334e+06,5724959.50,2.991251e+06,0.353631,2022-01-26,2022-03-28,61 days
2,2,239,243,4.063576e+06,3888887.25,-3.493783e+05,-0.042989,2022-05-11,2022-05-15,4 days
3,2,243,279,3.908137e+06,2780840.25,-2.254594e+06,-0.288449,2022-05-15,2022-06-20,36 days
4,3,279,350,2.794605e+06,2814323.25,5.915352e+04,0.007056,2022-06-20,2022-08-30,71 days
5,3,350,364,2.828254e+06,2911983.00,2.511865e+05,0.029604,2022-08-30,2022-09-14,15 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 490 configurations.
  output = _optimize_grid()


Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   41.917808
Equity Final [$]              13831751.419975
Equity Peak [$]               15220068.419975
Return [%]                          38.317514
Buy & Hold Return [%]              -43.967314
Return (Ann.) [%]                   38.317514
Volatility (Ann.) [%]               51.848523
Sharpe Ratio                         0.739028
Sortino Ratio                        1.699522
Calmar Ratio                         2.541827
Max. Drawdown [%]                  -15.074793
Avg. Drawdown [%]                   -6.245629
Max. Drawdown Duration       83 days 00:00:00
Avg. Drawdown Duration       18 days 00:00:00
# Trades                                    3
Win Rate [%]                            100.0
Best Trade [%]                      40.350776
Worst Trade [%]                      0.130227
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,2,132,196,4.148339e+06,5822226.50,3.347774e+06,0.403508,2022-01-24,2022-03-29,64 days
1,4,277,279,2.777224e+06,2780840.25,1.446683e+04,0.001302,2022-06-18,2022-06-20,2 days
2,4,279,364,2.794605e+06,2911983.00,4.695104e+05,0.042001,2022-06-20,2022-09-14,86 days


...

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-1,22,50,5.711706e+06,7208944.50,-1.497239e+06,-0.262135,2021-10-06,2021-11-03,28 days
1,-1,50,56,7.173260e+06,7650345.50,-4.770853e+05,-0.066509,2021-11-03,2021-11-09,6 days
2,-1,56,62,7.612476e+06,7468444.00,1.440323e+05,0.018921,2021-11-09,2021-11-15,6 days
3,-1,62,66,7.431475e+06,6502326.50,9.291487e+05,0.125029,2021-11-15,2021-11-19,4 days
4,1,66,70,6.534513e+06,6463256.00,-7.125702e+04,-0.010905,2021-11-19,2021-11-23,4 days
5,1,70,74,6.495249e+06,6089680.00,-4.055691e+05,-0.062441,2021-11-23,2021-11-27,4 days
6,1,74,81,6.119824e+06,6061149.50,-5.867442e+04,-0.009588,2021-11-27,2021-12-04,7 days
7,1,81,94,6.091152e+06,5416773.50,-6.743787e+05,-0.110714,2021-12-04,2021-12-17,13 days
8,1,94,107,5.443587e+06,5344083.50,-9.950303e+04,-0.018279,2021-12-17,2021-12-30,13 days
9,1,107,109,5.370537e+06,5330852.50,-3.968421e+04,-0.007389,2021-12-30,2022-01-01,2 days


c:\Python310\lib\site-packages\backtesting\backtesting.py:1488: UserWarning: Searching for best of 490 configurations.
  output = _optimize_grid()


Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                    87.39726
Equity Final [$]               30679027.95535
Equity Peak [$]                34149820.45535
Return [%]                          206.79028
Buy & Hold Return [%]              -43.967314
Return (Ann.) [%]                   206.79028
Volatility (Ann.) [%]              153.993336
Sharpe Ratio                         1.342852
Sortino Ratio                         7.46042
Calmar Ratio                        12.497731
Max. Drawdown [%]                  -16.546226
Avg. Drawdown [%]                   -4.529895
Max. Drawdown Duration       36 days 00:00:00
Avg. Drawdown Duration        9 days 00:00:00
# Trades                                    9
Win Rate [%]                        66.666667
Best Trade [%]                      54.720021
Worst Trade [%]                     -6.650885
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-1,46,50,7.061095e+06,7208944.50,-1.478493e+05,-0.020939,2021-10-30,2021-11-03,4 days
1,-1,50,56,7.173260e+06,7650345.50,-4.770853e+05,-0.066509,2021-11-03,2021-11-09,6 days
2,-1,56,62,7.612476e+06,7468444.00,1.440323e+05,0.018921,2021-11-09,2021-11-15,6 days
3,-1,62,131,7.431475e+06,3984710.50,3.446765e+06,0.463806,2021-11-15,2022-01-23,69 days
4,3,131,195,4.004435e+06,5724959.50,5.161574e+06,0.429655,2022-01-23,2022-03-28,64 days
5,-3,195,200,5.696621e+06,5670297.50,7.897035e+04,0.004621,2022-03-28,2022-04-02,5 days
6,-3,200,202,5.642230e+06,5695885.50,-1.609679e+05,-0.009510,2022-04-02,2022-04-04,2 days
7,-3,202,278,5.667691e+06,2566329.25,9.304085e+06,0.547200,2022-04-04,2022-06-19,76 days
8,10,278,364,2.579033e+06,2911983.00,3.329504e+06,0.129099,2022-06-19,2022-09-14,87 days


<Strategy RsiOscillator_WithShortPosition(upper_bound=60,lower_bound=30,rsi_window=26)>

**ドンチャンブレイクアウト**

In [79]:
import BacktestingStrategies.Strategy_Donchian as btstdnchn
importlib.reload(btstdnchn)

# バックテストを設定
bt = Backtest(
    data, # チャートデータ
    btstdnchn.DnchnBreakout, # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True #自動でポジションをクローズ(オープン)
)

output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + ".html")

# #最適化
p = psutil.Process()
p.nice(psutil.HIGH_PRIORITY_CLASS)

output2=bt.optimize(\
    dnchn_long = range(10, 50, 5),
    dnchn_short = range(10, 50, 5),
    maximize='Equity Final [$]',\
    method='grid',\
    constraint = lambda x: (x.dnchn_long > x.dnchn_short),\
    max_tries = 1000)
print(output2)
display(output2._trades)
display(output2._strategy)
bt.plot(filename = fname + str(output._strategy) + "_optimize.html") 

if position == 1:
    # バックテストを設定
    bt = Backtest(
        data, # チャートデータ
        btstdnchn.DnchnBreakout_WithShortPosition, # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )

    output = bt.run() # バックテスト実行
    print(output) # 実行結果(データ)
    display(output._trades)
    bt.plot(filename = fname + str(output._strategy) + ".html")

    # #最適化
    p = psutil.Process()
    p.nice(psutil.HIGH_PRIORITY_CLASS)

    output2=bt.optimize(\
        dnchn_long = range(10, 50, 5),
        dnchn_short = range(10, 50, 5),
        maximize='Equity Final [$]',\
        method='grid',\
        max_tries = 1000)
    print(output2)
    display(output2._trades)
    display(output2._strategy)
    bt.plot(filename = fname + str(output._strategy) + "_optimize.html")

Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   26.575342
Equity Final [$]                 8180229.3919
Equity Peak [$]                 11646166.7865
Return [%]                         -18.197706
Buy & Hold Return [%]              -43.967314
Return (Ann.) [%]                  -18.197706
Volatility (Ann.) [%]               16.047082
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -29.760328
Avg. Drawdown [%]                   -8.591448
Max. Drawdown Duration      329 days 00:00:00
Avg. Drawdown Duration       86 days 00:00:00
# Trades                                   19
Win Rate [%]                        57.894737
Best Trade [%]                       7.500997
Worst Trade [%]                    -15.424754
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,1,22,23,5.768533e+06,6165174.5,3.966414e+05,0.068759,2021-10-06,2021-10-07,1 days
1,1,23,28,6.195692e+06,6522933.0,3.272409e+05,0.052817,2021-10-07,2021-10-12,5 days
2,1,28,32,6.555222e+06,7046928.5,4.917070e+05,0.075010,2021-10-12,2021-10-16,4 days
3,1,32,36,7.081811e+06,7363574.5,2.817637e+05,0.039787,2021-10-16,2021-10-20,4 days
4,1,36,37,7.400024e+06,7549913.0,1.498888e+05,0.020255,2021-10-20,2021-10-21,1 days
5,1,37,44,7.587285e+06,6652332.0,-9.349531e+05,-0.123226,2021-10-21,2021-10-28,7 days
6,1,56,66,7.688215e+06,6502326.5,-1.185888e+06,-0.154248,2021-11-09,2021-11-19,10 days
7,1,147,157,5.071106e+06,4657804.5,-4.133013e+05,-0.081501,2022-02-08,2022-02-18,10 days
8,1,191,192,5.220480e+06,5379133.5,1.586539e+05,0.030391,2022-03-24,2022-03-25,1 days
9,1,192,193,5.405760e+06,5414452.5,8.692289e+03,0.001608,2022-03-25,2022-03-26,1 days


Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   25.205479
Equity Final [$]                 8683936.2855
Equity Peak [$]                10988355.92245
Return [%]                         -13.160637
Buy & Hold Return [%]              -43.967314
Return (Ann.) [%]                  -13.160637
Volatility (Ann.) [%]               15.720162
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -20.971469
Avg. Drawdown [%]                   -8.132854
Max. Drawdown Duration      310 days 00:00:00
Avg. Drawdown Duration       84 days 00:00:00
# Trades                                   12
Win Rate [%]                        66.666667
Best Trade [%]                       7.500997
Worst Trade [%]                    -15.424754
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,1,28,32,6.555222e+06,7046928.5,4.917070e+05,0.075010,2021-10-12,2021-10-16,4 days
1,1,32,36,7.081811e+06,7363574.5,2.817637e+05,0.039787,2021-10-16,2021-10-20,4 days
2,1,36,37,7.400024e+06,7549913.0,1.498888e+05,0.020255,2021-10-20,2021-10-21,1 days
3,1,37,56,7.587285e+06,7650345.5,6.306043e+04,0.008311,2021-10-21,2021-11-09,19 days
4,1,56,66,7.688215e+06,6502326.5,-1.185888e+06,-0.154248,2021-11-09,2021-11-19,10 days
5,1,192,193,5.405760e+06,5414452.5,8.692289e+03,0.001608,2022-03-25,2022-03-26,1 days
6,1,193,195,5.441254e+06,5724959.5,2.837055e+05,0.052140,2022-03-26,2022-03-28,2 days
7,1,195,196,5.753298e+06,5822226.5,6.892845e+04,0.011981,2022-03-28,2022-03-29,1 days
8,1,196,210,5.851047e+06,4960690.5,-8.903560e+05,-0.152170,2022-03-29,2022-04-12,14 days
9,2,308,309,3.121799e+06,3232717.0,2.218369e+05,0.035530,2022-07-19,2022-07-20,1 days


<Strategy DnchnBreakout(dnchn_long=25,dnchn_short=15)>

Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   93.972603
Equity Final [$]               7733712.162987
Equity Peak [$]               11937064.674125
Return [%]                         -22.662878
Buy & Hold Return [%]              -43.967314
Return (Ann.) [%]                  -22.662878
Volatility (Ann.) [%]               45.705054
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -35.683848
Avg. Drawdown [%]                  -15.100385
Max. Drawdown Duration      241 days 00:00:00
Avg. Drawdown Duration       69 days 00:00:00
# Trades                                   50
Win Rate [%]                             68.0
Best Trade [%]                      15.843426
Worst Trade [%]                    -27.267215
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,1,22,23,5.768533e+06,6165174.50,3.966414e+05,0.068759,2021-10-06,2021-10-07,1 days
1,1,23,28,6.195692e+06,6522933.00,3.272409e+05,0.052817,2021-10-07,2021-10-12,5 days
2,1,28,32,6.555222e+06,7046928.50,4.917070e+05,0.075010,2021-10-12,2021-10-16,4 days
3,1,32,36,7.081811e+06,7363574.50,2.817637e+05,0.039787,2021-10-16,2021-10-20,4 days
4,1,36,37,7.400024e+06,7549913.00,1.498888e+05,0.020255,2021-10-20,2021-10-21,1 days
5,1,37,44,7.587285e+06,6652332.00,-9.349531e+05,-0.123226,2021-10-21,2021-10-28,7 days
6,-1,44,56,6.619403e+06,7650345.50,-1.030943e+06,-0.155746,2021-10-28,2021-11-09,12 days
7,1,56,66,7.688215e+06,6502326.50,-1.185888e+06,-0.154248,2021-11-09,2021-11-19,10 days
8,-1,66,74,6.470140e+06,6089680.00,3.804600e+05,0.058802,2021-11-19,2021-11-27,8 days
9,-1,74,81,6.059536e+06,6061149.50,-1.613416e+03,-0.000266,2021-11-27,2021-12-04,7 days


Start                     2021-09-14 00:00:00
End                       2022-09-14 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   84.657534
Equity Final [$]              11128416.727937
Equity Peak [$]               13136447.236075
Return [%]                          11.284167
Buy & Hold Return [%]              -43.967314
Return (Ann.) [%]                   11.284167
Volatility (Ann.) [%]               72.927667
Sharpe Ratio                         0.154731
Sortino Ratio                         0.28155
Calmar Ratio                         0.347521
Max. Drawdown [%]                  -32.470494
Avg. Drawdown [%]                  -16.380086
Max. Drawdown Duration      145 days 00:00:00
Avg. Drawdown Duration       61 days 00:00:00
# Trades                                   35
Win Rate [%]                        74.285714
Best Trade [%]                      15.843426
Worst Trade [%]                    -27.241051
Avg. Trade [%]                    

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,1,56,66,7.688215e+06,6502326.50,-1.185888e+06,-0.154248,2021-11-09,2021-11-19,10 days
1,-1,66,74,6.470140e+06,6089680.00,3.804600e+05,0.058802,2021-11-19,2021-11-27,8 days
2,-1,74,81,6.059536e+06,6061149.50,-1.613416e+03,-0.000266,2021-11-27,2021-12-04,7 days
3,-1,81,82,6.031147e+06,5550177.50,4.809693e+05,0.079748,2021-12-04,2021-12-05,1 days
4,-1,82,114,5.522704e+06,5059130.00,4.635741e+05,0.083940,2021-12-05,2022-01-06,32 days
5,-2,114,116,5.034087e+06,4801785.00,4.646046e+05,0.046146,2022-01-06,2022-01-08,2 days
6,-2,116,130,4.778016e+06,4146638.25,1.262756e+06,0.132142,2022-01-08,2022-01-22,14 days
7,-2,130,131,4.126112e+06,3984710.50,2.828038e+05,0.034270,2022-01-22,2022-01-23,1 days
8,-3,131,157,3.964986e+06,4657804.50,-2.078455e+06,-0.174734,2022-01-23,2022-02-18,26 days
9,-2,157,158,4.634748e+06,4602393.00,6.471074e+04,0.006981,2022-02-18,2022-02-19,1 days


<Strategy DnchnBreakout_WithShortPosition(dnchn_long=45,dnchn_short=10)>